# Case Study 1 - fitting spectral data cubes with pPXF

If you do not already have pPXF installed, you can do so with `pip install ppxf`.
The files produced in this notebook are then used for comparison with the kinematic SimSpin files produced in the R code files run previously. 

In [1]:
import numpy as np
from astropy.io import fits
from astropy.coordinates import Angle
import matplotlib.pyplot as plt
from scipy.stats import scoreatpercentile
from scipy import ndimage,signal
import glob
import os
import sys

from matplotlib.ticker import MultipleLocator
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.ticker as ticker

from ppxf.ppxf import ppxf, rebin
import ppxf.ppxf_util as util
from ppxf import miles_util as lib

import threading
from argparse import ArgumentParser

c = 299792.458 # speed of light in km/s

In [2]:
#### VARIABLES ####
root_in            = "/cubes/"
root_out           = "/cubes/ppxf_output/"
template_loc       = "/data/EMILES_PADOVA_CH/Ech1.30*.fits"

file_name_list     = ["disk_EMILES_spectral_fwhm0_lowz",
                      "bulge_EMILES_spectral_fwhm0_lowz",
                      "old_bulge_EMILES_spectral_fwhm0_lowz",
                      "disk_BC03hr_spectral_fwhm0_lowz",
                      "bulge_BC03hr_spectral_fwhm0_lowz",
                      "old_bulge_BC03hr_spectral_fwhm0_lowz"]
folder_list        = ["fwhm0_lowz/disk_EMILES/",
                      "fwhm0_lowz/bulge_EMILES/",
                      "fwhm0_lowz/old_bulge_EMILES/",
                      "fwhm0_lowz/disk_BC03hr/",
                      "fwhm0_lowz/bulge_BC03hr/",
                      "fwhm0_lowz/old_bulge_BC03hr/"]

redshift           = 0.0144
telescope_FWHM_list= [0, 0, 0, 0, 0, 0]
template_FWHM_list = [2.51, 2.51, 2.51, 3, 3, 3]
n_mom              = 4
plot               = True


In [3]:
for i in range(1,3):
    
    folder = folder_list[i]
    template_FWHM = template_FWHM_list[i]
    file_name = file_name_list[i]
    telescope_FWHM = telescope_FWHM_list[i]
    FWHM_temp = 2.51
    z = redshift
        
    #### COMPUTATION based on variables ####
    if telescope_FWHM <= template_FWHM:
        FWHM_gal = (template_FWHM * (1 + z)) # Angstroms (resolution for this "observed" template)
    else:
        FWHM_gal = telescope_FWHM

    file_root = f"{root_in}{folder}"
    ppxf_out_root = f"{root_out}{folder}"
    #########################################

    inputFileExists = os.path.exists(f"{file_root}{file_name}.FITS")
    outputDirectoryExists = os.path.exists(ppxf_out_root)

    if not inputFileExists or not outputDirectoryExists:
        print("\nError - check you file names before proceading.")

        if not inputFileExists:
            print(f"Input FITS file (",f"{file_root}{file_name}.FITS", ") does not exist. \n")
            sys.exit()
        else:
            print("Output directory (",f"{ppxf_out_root}",") for pPXF files does not exist. \n")
            sys.exit()

    # Check n_mom is 2 or 4

    if n_mom != 2. and n_mom != 4.:
        print("Number of requested velocity moments must be 2 OR 4.")
        sys.exit()

    # Make a new pPXF fit folder at the location of ppxf_out_root for fitting plots
    try:
      os.makedirs(f"{ppxf_out_root}/pPXF_fits", exist_ok=True)
    except OSError as error:
      print("Invalid path name for output pPXF files. \n")
      sys.exit()
        
    #---------------------------------#
    # Step 1: Read in the observation #
    #---------------------------------#

    hdu = fits.open(f"{file_root}{file_name}.FITS")

    cube     = hdu["DATA"].data
    head_obs = hdu["DATA"].header
    var      = hdu["STAT"].data*1e40
    
    npart    = hdu["NPART"].data
    npart_flat = npart.reshape(-1)
    npart_flat[npart_flat == 0] = None

    hdu.close()
    
    print(f"Reading spectral cube of size {cube.shape}.")

    # Reshape the cube into an array
    npix = cube.shape[0]
    nspec = cube.shape[1] * cube.shape[2]
    spectrum = cube.reshape(npix, -1) # create an array of spectra [npix, (sbin_x * sbin_y)]
    noise_flat = var.reshape(npix, -1) 

    print(f"Re-shaping cube into array of shape {spectrum.shape}, with {spectrum.shape[0]} wavelengths and {spectrum.shape[1]} pixels.")
    print(f"Re-shaping noise into array of shape {noise_flat.shape}, with {noise_flat.shape[0]} wavelengths and {noise_flat.shape[1]} pixels.")

    # Begin by pulling out the wavelength axis of the cube -------------------------------------------------------------
    # These properties are necessary for logarithmically re-binning the spectra before
    # fitting with pPXF.
    wave_axis = head_obs['CRVAL3'] + (np.arange(0., (head_obs['NAXIS3']))*head_obs['CDELT3'])
    wave_range = head_obs["CRVAL3"] + np.array([0., head_obs["CDELT3"]*(head_obs["NAXIS3"]-1)])
    velscale = c * np.diff(np.log(wave_axis[-2:])) # Smallest velocity step
    velscale = velscale[0]
    
    print(f"CRVAL3: {head_obs['CRVAL3']}, CRDELT3: {head_obs['CDELT3']} and NAXIS3:{head_obs['NAXIS3']}")

    print(f"The velocity scale of the `observed` spectrum is {velscale} km/s per pixel.")

    # We also need to bring the observed spectrum back to the rest-frame ------------------------------------
    if z >= 0.0:
        wave_range = wave_range / (1 + z)
        FWHM_gal = FWHM_gal / (1 + z)
        z = 0
        print(f"The wavelength range of the 'observed' galaxy (returned to rest-frame) is {wave_range} Angstrom.")
        print(f"The FWHM at this restframe is {FWHM_gal}.")

    else:
        print(f"The wavelength range of the 'observed' galaxy is {wave_range} Angstrom.")
        print(f"The FWHM of the observation is {FWHM_gal}.")

    #---------------------------------#
    # Step 2: Read in the templates   #
    #---------------------------------#
    # Next, load the templates necessary to perform the fit with pPXF

    # Using pPXF function to read in the templates located at `template_loc`, this
    # function reads each template spectrum into an array sorted by age and metallicity.
    # If the FWHM_gal is larger than the FWHM_temp, this function will also convolve
    # the templates down to the same resolution as the observations.

    print(f"The FWHM of the templates is {FWHM_temp}.")
    if FWHM_gal <= FWHM_temp:
        FWHM_gal = None
        FWHM_temp = None
        print(f"No convolution can be made. Setting FWHM_gal, FWHM_temp = None.")

    miles = lib.miles(pathname = template_loc, velscale = velscale, FWHM_gal = FWHM_gal, FWHM_tem = FWHM_temp, norm_range=[5000, 5950])

    templates = miles.templates
    stars_templates = templates.reshape(templates.shape[0], -1)
    stars_templates /= np.median(stars_templates) # Normalizes stellar templates by a scalar

    log_lam_temp = miles.ln_lam_temp
    #reg_dim = miles.templates.shape[1:]
    #regul_err = 0.013  # Desired regularization error

    mask = (np.exp(log_lam_temp) > (np.min(wave_range)-500)) & (np.exp(log_lam_temp) < (np.max(wave_range)+500))

    log_lam_temp = log_lam_temp[mask]
    stars_templates = stars_templates[mask]

    wave_range_temp =[np.exp(np.min(log_lam_temp)), np.exp(np.max(log_lam_temp))]

    print(f"The wavelength range of the template spectra is {wave_range_temp} Angstrom.")
    print(f"Array of templates containing shape {stars_templates.shape}.")

    #----------------------------------------#
    # Step 3: Fitting the observed spectra   #
    #----------------------------------------#

    # Preparing the galaxy observations for input into pPXF
    galaxy, log_lam_galaxy, velscale = util.log_rebin(wave_range, spectrum[:,11], velscale=velscale)
    galaxy /= np.median(galaxy)

    dv = (log_lam_temp[0] - log_lam_galaxy[0])*c
    vel = c*np.log(1 + z)
    start = [vel, 200.] # starting guess for kinematic measurements

    print(f"The starting guess velocity, as according to Cappellari 2017 [V = c*log(1 + z)] is {vel} km/s.")

    # Now fitting each pixel in turn and comparing to the LOS_VEL and LOS_DISP values ----------------------------------
    if n_mom == 2:
        ppxf_velocity    = np.empty_like(npart_flat)
        ppxf_dispersion  = np.empty_like(npart_flat)
        ppxf_chi2        = np.empty_like(npart_flat)
    else:
        ppxf_velocity    = np.empty_like(npart_flat)
        ppxf_dispersion  = np.empty_like(npart_flat)
        ppxf_h3          = np.empty_like(npart_flat)
        ppxf_h4          = np.empty_like(npart_flat)
        ppxf_chi2        = np.empty_like(npart_flat)

    print(f"The 'observed' spectrum has a staring wavelength of {log_lam_galaxy[0]} Angstrom.")
    print(f"The template spectrum has a staring wavelength of {log_lam_temp[0]} Angstrom.")
    print(f"The difference is therefore {dv} km/s.")

    for j in range(nspec):
        if npart_flat[j] > 0:
            print(f"Fitting pixel [{j}]...")
            galaxy, log_lam_galaxy, velscale = util.log_rebin(wave_range, spectrum[:,j], velscale=velscale)
            noise_rebin, log_lam_noise, velscale_2 = util.log_rebin(wave_range, 1/np.sqrt(noise_flat[:,j]), velscale=velscale)
            noise_rebin /= np.median(galaxy)
            galaxy /= np.median(galaxy)
            
            if n_mom == 2:
                pp = ppxf(stars_templates, galaxy, noise_rebin, velscale, start,
                          vsyst=dv, lam = np.exp(log_lam_galaxy),
                          plot=False, moments=2, degree=-1, mdegree=7,  # Set plot to False when using in batch mode. change mdegree=10 for testing
                          clean=False, regul=False)
                ppxf_velocity[j]   = pp.sol[0]
                ppxf_dispersion[j] = pp.sol[1]
                ppxf_chi2[j]       = pp.chi2

                if plot:
                  pp.plot()
                  plt.savefig(f"{ppxf_out_root}/pPXF_fits/pPXF_fit_{j}.png")
                  plt.close()

            else:
                pp = ppxf(stars_templates, galaxy, noise_rebin, velscale, start,
                          vsyst=dv, lam = np.exp(log_lam_galaxy), 
                          plot=False, moments=4, degree=-1, mdegree=7,  # Set plot to False when using in batch mode. change mdegree=10 for testing
                          clean=False, regul=False)
                
                ppxf_velocity[j]   = pp.sol[0]
                ppxf_dispersion[j] = pp.sol[1]
                ppxf_h3[j]         = pp.sol[2]
                ppxf_h4[j]         = pp.sol[3]
                ppxf_chi2[j]       = pp.chi2

                if plot:
                  pp.plot()
                  plt.savefig(f"{ppxf_out_root}/pPXF_fits/pPXF_fit_{j}.png")
                  plt.close()


        else:
            if n_mom == 2:
                print(f"Pixel [{j}] is empty, proceading to next one...")
                ppxf_velocity[j]   = np.nan
                ppxf_dispersion[j] = np.nan
                ppxf_chi2[j]       = np.nan

            else:
                print(f"Pixel [{j}] is empty, proceading to next one...")
                ppxf_velocity[j]   = np.nan
                ppxf_dispersion[j] = np.nan
                ppxf_h3[j]         = np.nan
                ppxf_h4[j]         = np.nan
                ppxf_chi2[j]       = np.nan

    # --------------------------------- #
    # Step 4: Reshaping the fit results #
    #         back to images            #
    # --------------------------------- #

    fit_velocity   = ppxf_velocity.reshape([cube.shape[1], cube.shape[2]])
    fit_dispersion = ppxf_dispersion.reshape([cube.shape[1], cube.shape[2]])
    fit_chi2       = ppxf_chi2.reshape([cube.shape[1], cube.shape[2]])

    np.save(f"{ppxf_out_root}{file_name}_ppxf_velocity.npy", fit_velocity)
    np.save(f"{ppxf_out_root}{file_name}_ppxf_dispersion.npy", fit_dispersion)
    np.save(f"{ppxf_out_root}{file_name}_ppxf_chi2.npy", fit_chi2)

    if n_mom == 4:

        fit_h3 = ppxf_h3.reshape([cube.shape[1], cube.shape[2]])
        fit_h4 = ppxf_h4.reshape([cube.shape[1], cube.shape[2]])
        np.save(f"{ppxf_out_root}{file_name}_ppxf_h3.npy", fit_h3)
        np.save(f"{ppxf_out_root}{file_name}_ppxf_h4.npy", fit_h4)
        
    print(f"Done with the {i} iteration.")
    del cube, galaxy, noise_rebin, var, head_obs


Reading spectral cube of size (1924, 30, 30).
Re-shaping cube into array of shape (1924, 900), with 1924 wavelengths and 900 pixels.
Re-shaping noise into array of shape (1924, 900), with 1924 wavelengths and 900 pixels.
CRVAL3: 3750, CRDELT3: 1.04 and NAXIS3:1924
The velocity scale of the `observed` spectrum is 54.228990358951044 km/s per pixel.
The wavelength range of the 'observed' galaxy (returned to rest-frame) is [3696.76656151 5668.29652997] Angstrom.
The FWHM at this restframe is 2.51.
The FWHM of the templates is 2.51.
No convolution can be made. Setting FWHM_gal, FWHM_temp = None.
The wavelength range of the template spectra is [3196.81691845587, 6167.683415503795] Angstrom.
Array of templates containing shape (3634, 258).
The starting guess velocity, as according to Cappellari 2017 [V = c*log(1 + z)] is 0.0 km/s.
The 'observed' spectrum has a staring wavelength of 8.215165582202367 Angstrom.
The template spectrum has a staring wavelength of 8.06991088075 Angstrom.
The differ

 Best Fit:       Vel     sigma        h3        h4
 comp.  0:       -15       105     0.045    -0.024
chi2/DOF: 1.033; DOF: 2353; degree = -1; mdegree = 7
method = capfit; Jac calls: 11; Func calls: 135; Status: 2
linear_method = lsq_box; Nonzero Templates (>0.1%): 7/258
Fitting pixel [51]...
 Best Fit:       Vel     sigma        h3        h4
 comp.  0:       -12        88    -0.018     0.004
chi2/DOF: 0.8667; DOF: 2353; degree = -1; mdegree = 7
method = capfit; Jac calls: 7; Func calls: 90; Status: 2
linear_method = lsq_box; Nonzero Templates (>0.1%): 7/258
Pixel [52] is empty, proceading to next one...
Pixel [53] is empty, proceading to next one...
Pixel [54] is empty, proceading to next one...
Pixel [55] is empty, proceading to next one...
Pixel [56] is empty, proceading to next one...
Pixel [57] is empty, proceading to next one...
Pixel [58] is empty, proceading to next one...
Pixel [59] is empty, proceading to next one...
Pixel [60] is empty, proceading to next one...
Pixel [61] i

 Best Fit:       Vel     sigma        h3        h4
 comp.  0:        -9        59     0.010     0.264
chi2/DOF: 1.019; DOF: 2353; degree = -1; mdegree = 7
method = capfit; Jac calls: 14; Func calls: 172; Status: 2
linear_method = lsq_box; Nonzero Templates (>0.1%): 7/258
Fitting pixel [102]...
 Best Fit:       Vel     sigma        h3        h4
 comp.  0:         1        88    -0.053     0.028
chi2/DOF: 1.196; DOF: 2353; degree = -1; mdegree = 7
method = capfit; Jac calls: 11; Func calls: 135; Status: 2
linear_method = lsq_box; Nonzero Templates (>0.1%): 5/258
Fitting pixel [103]...
 Best Fit:       Vel     sigma        h3        h4
 comp.  0:        -5        73    -0.045     0.138
chi2/DOF: 1.148; DOF: 2353; degree = -1; mdegree = 7
method = capfit; Jac calls: 9; Func calls: 114; Status: 2
linear_method = lsq_box; Nonzero Templates (>0.1%): 7/258
Fitting pixel [104]...
 Best Fit:       Vel     sigma        h3        h4
 comp.  0:       -19        64     0.122     0.300
chi2/DOF: 1.10

 Best Fit:       Vel     sigma        h3        h4
 comp.  0:        -7        62     0.055     0.300
chi2/DOF: 0.9395; DOF: 2353; degree = -1; mdegree = 7
method = capfit; Jac calls: 7; Func calls: 91; Status: 2
linear_method = lsq_box; Nonzero Templates (>0.1%): 9/258
Fitting pixel [138]...
 Best Fit:       Vel     sigma        h3        h4
 comp.  0:        -6        94    -0.081    -0.016
chi2/DOF: 1.197; DOF: 2353; degree = -1; mdegree = 7
method = capfit; Jac calls: 12; Func calls: 150; Status: 2
linear_method = lsq_box; Nonzero Templates (>0.1%): 9/258
Fitting pixel [139]...
 Best Fit:       Vel     sigma        h3        h4
 comp.  0:       -10       101    -0.002    -0.016
chi2/DOF: 1.120; DOF: 2353; degree = -1; mdegree = 7
method = capfit; Jac calls: 14; Func calls: 175; Status: 2
linear_method = lsq_box; Nonzero Templates (>0.1%): 8/258
Fitting pixel [140]...
 Best Fit:       Vel     sigma        h3        h4
 comp.  0:        -7       100    -0.004    -0.031
chi2/DOF: 1.33

 Best Fit:       Vel     sigma        h3        h4
 comp.  0:       -22        94     0.051     0.048
chi2/DOF: 0.9121; DOF: 2353; degree = -1; mdegree = 7
method = capfit; Jac calls: 9; Func calls: 112; Status: 2
linear_method = lsq_box; Nonzero Templates (>0.1%): 5/258
Fitting pixel [172]...
 Best Fit:       Vel     sigma        h3        h4
 comp.  0:        -5        88     0.034     0.022
chi2/DOF: 0.9718; DOF: 2353; degree = -1; mdegree = 7
method = capfit; Jac calls: 9; Func calls: 110; Status: 2
linear_method = lsq_box; Nonzero Templates (>0.1%): 7/258
Fitting pixel [173]...
 Best Fit:       Vel     sigma        h3        h4
 comp.  0:       -19        65     0.114     0.300
chi2/DOF: 1.048; DOF: 2353; degree = -1; mdegree = 7
method = capfit; Jac calls: 11; Func calls: 139; Status: 2
linear_method = lsq_box; Nonzero Templates (>0.1%): 8/258
Fitting pixel [174]...
 Best Fit:       Vel     sigma        h3        h4
 comp.  0:         1        94     0.011     0.005
chi2/DOF: 0.8

...

 Best Fit:       Vel     sigma        h3        h4
 comp.  0:        -4       100    -0.017     0.013
chi2/DOF: 0.8379; DOF: 2353; degree = -1; mdegree = 7
method = capfit; Jac calls: 9; Func calls: 113; Status: 2
linear_method = lsq_box; Nonzero Templates (>0.1%): 7/258
Fitting pixel [298]...
 Best Fit:       Vel     sigma        h3        h4
 comp.  0:       -14        91    -0.000     0.029
chi2/DOF: 0.8454; DOF: 2353; degree = -1; mdegree = 7
method = capfit; Jac calls: 8; Func calls: 98; Status: 2
linear_method = lsq_box; Nonzero Templates (>0.1%): 10/258
Pixel [299] is empty, proceading to next one...
Pixel [300] is empty, proceading to next one...
Fitting pixel [301]...
 Best Fit:       Vel     sigma        h3        h4
 comp.  0:        12        83    -0.143     0.071
chi2/DOF: 0.9860; DOF: 2353; degree = -1; mdegree = 7
method = capfit; Jac calls: 9; Func calls: 110; Status: 2
linear_method = lsq_box; Nonzero Templates (>0.1%): 7/258
Fitting pixel [302]...
 Best Fit:       Ve

 Best Fit:       Vel     sigma        h3        h4
 comp.  0:        -2        94    -0.046     0.025
chi2/DOF: 0.8990; DOF: 2353; degree = -1; mdegree = 7
method = capfit; Jac calls: 6; Func calls: 74; Status: 2
linear_method = lsq_box; Nonzero Templates (>0.1%): 6/258
Fitting pixel [328]...
 Best Fit:       Vel     sigma        h3        h4
 comp.  0:        -6       113     0.036    -0.065
chi2/DOF: 1.368; DOF: 2353; degree = -1; mdegree = 7
method = capfit; Jac calls: 7; Func calls: 86; Status: 2
linear_method = lsq_box; Nonzero Templates (>0.1%): 11/258
Pixel [329] is empty, proceading to next one...
Fitting pixel [330]...
 Best Fit:       Vel     sigma        h3        h4
 comp.  0:       -12        69     0.015     0.300
chi2/DOF: 1.028; DOF: 2353; degree = -1; mdegree = 7
method = capfit; Jac calls: 9; Func calls: 113; Status: 2
linear_method = lsq_box; Nonzero Templates (>0.1%): 5/258
Fitting pixel [331]...
 Best Fit:       Vel     sigma        h3        h4
 comp.  0:       -1

 Best Fit:       Vel     sigma        h3        h4
 comp.  0:       -13        84     0.023     0.039
chi2/DOF: 0.9203; DOF: 2353; degree = -1; mdegree = 7
method = capfit; Jac calls: 7; Func calls: 86; Status: 2
linear_method = lsq_box; Nonzero Templates (>0.1%): 7/258
Fitting pixel [357]...
 Best Fit:       Vel     sigma        h3        h4
 comp.  0:       -17        80     0.006     0.095
chi2/DOF: 1.022; DOF: 2353; degree = -1; mdegree = 7
method = capfit; Jac calls: 8; Func calls: 103; Status: 2
linear_method = lsq_box; Nonzero Templates (>0.1%): 6/258
Fitting pixel [358]...
 Best Fit:       Vel     sigma        h3        h4
 comp.  0:         0       101    -0.028    -0.036
chi2/DOF: 0.9318; DOF: 2353; degree = -1; mdegree = 7
method = capfit; Jac calls: 26; Func calls: 320; Status: 2
linear_method = lsq_box; Nonzero Templates (>0.1%): 9/258
Fitting pixel [359]...
 Best Fit:       Vel     sigma        h3        h4
 comp.  0:        -9        82    -0.101     0.033
chi2/DOF: 1.29

 Best Fit:       Vel     sigma        h3        h4
 comp.  0:       -11        84     0.024     0.056
chi2/DOF: 1.231; DOF: 2353; degree = -1; mdegree = 7
method = capfit; Jac calls: 7; Func calls: 86; Status: 2
linear_method = lsq_box; Nonzero Templates (>0.1%): 12/258
Fitting pixel [385]...
 Best Fit:       Vel     sigma        h3        h4
 comp.  0:       -15        92     0.048    -0.007
chi2/DOF: 0.9781; DOF: 2353; degree = -1; mdegree = 7
method = capfit; Jac calls: 6; Func calls: 74; Status: 2
linear_method = lsq_box; Nonzero Templates (>0.1%): 8/258
Fitting pixel [386]...
 Best Fit:       Vel     sigma        h3        h4
 comp.  0:       -11        95    -0.005     0.035
chi2/DOF: 0.8641; DOF: 2353; degree = -1; mdegree = 7
method = capfit; Jac calls: 9; Func calls: 110; Status: 2
linear_method = lsq_box; Nonzero Templates (>0.1%): 10/258
Fitting pixel [387]...
 Best Fit:       Vel     sigma        h3        h4
 comp.  0:        -4        85     0.006     0.043
chi2/DOF: 0.98

KeyboardInterrupt: 